# Yoga Pose Classification

### 1.Introduction

### 2.Motivation

### 3.Problem Statement

### 4.Data Sources：

### 5.Dataset Description:

### 6.Challenging Aspects:
The Yoga-82 dataset presents several challenges:


### Method 1: CNN + MobileNetV3

#### 1.1 Import Necessary Libraries

In [1]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
import random

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
from PIL import Image, ImageFile

# Tensorflow Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint,TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import Model
# from tensorflow.keras.layers import Rescaling, Normalization, RandomFlip, RandomRotation
from tensorflow.keras.layers.experimental import preprocessing

# 检查 GPU 是否可用
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("Using GPU:", tf.config.list_physical_devices('GPU')[0])
else:
    print("No GPU detected, using CPU instead.")


# System libraries
from pathlib import Path
import os.path
import requests
import sys
import os

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

Num GPUs Available:  1
Using GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


#### 1.2 Create Helper Functions

In [2]:
# 设置文件 URL
url = "https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py"

file_path = "helper_functions.py"

# Check if the file already exists
if not os.path.exists(file_path):
    # Send a GET request to fetch the file content
    response = requests.get(url)
    
    # Write the content to the file
    with open(file_path, "wb") as file:
        file.write(response.content)
    
    print("Download complete!")
else:
    print("helper_functions.py already exists locally. Skipping download.")

# Add the current working directory to the Python path
sys.path.append(os.getcwd())

# Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

helper_functions.py already exists locally. Skipping download.


### 1.3 Load and Transform Data

In [3]:
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

# Walk through each directory
dataset = "../yoga_dataset_links"
walk_through_dir(dataset)

There are 82 directories and 0 images in '../yoga_dataset_links'.
There are 0 directories and 84 images in '../yoga_dataset_links\Akarna_Dhanurasana'.
There are 0 directories and 76 images in '../yoga_dataset_links\Bharadvaja's_Twist_pose_or_Bharadvajasana_I_'.
There are 0 directories and 372 images in '../yoga_dataset_links\Boat_Pose_or_Paripurna_Navasana_'.
There are 0 directories and 260 images in '../yoga_dataset_links\Bound_Angle_Pose_or_Baddha_Konasana_'.
There are 0 directories and 216 images in '../yoga_dataset_links\Bow_Pose_or_Dhanurasana_'.
There are 0 directories and 251 images in '../yoga_dataset_links\Bridge_Pose_or_Setu_Bandha_Sarvangasana_'.
There are 0 directories and 307 images in '../yoga_dataset_links\Camel_Pose_or_Ustrasana_'.
There are 0 directories and 404 images in '../yoga_dataset_links\Cat_Cow_Pose_or_Marjaryasana_'.
There are 0 directories and 302 images in '../yoga_dataset_links\Chair_Pose_or_Utkatasana_'.
There are 0 directories and 293 images in '../yoga_d

### 1.4 Placing Data into a Dataframe

In [4]:
image_dir = Path(dataset)

# Get filepaths and labels
filepaths = list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.png')) + list(image_dir.glob(r'**/*.png'))

labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

# Concatenate filepaths and labels
image_df = pd.concat([filepaths, labels], axis=1)

image_df

,Filepath,Label
0,..\yoga_dataset_links\Akarna_Dhanurasana\0_15.jpg,Akarna_Dhanurasana
1,..\yoga_dataset_links\Akarna_Dhanurasana\0_162...,Akarna_Dhanurasana
2,..\yoga_dataset_links\Akarna_Dhanurasana\0_18.jpg,Akarna_Dhanurasana
3,..\yoga_dataset_links\Akarna_Dhanurasana\0_305...,Akarna_Dhanurasana
4,..\yoga_dataset_links\Akarna_Dhanurasana\0_377...,Akarna_Dhanurasana
...,...,...
37195,..\yoga_dataset_links\Yogic_sleep_pose\2_16.jpg,Yogic_sleep_pose
37196,..\yoga_dataset_links\Yogic_sleep_pose\2_20.jpg,Yogic_sleep_pose
37197,..\yoga_dataset_links\Yogic_sleep_pose\2_8.jpg,Yogic_sleep_pose
37198,..\yoga_dataset_links\Cobra_Pose_or_Bhujangasa...,Cobra_Pose_or_Bhujangasana_


### 1.7 Data Preprocessing
<p style="font-size:15px; font-family:verdana; line-height: 1.7em">The data will be split into three different categories: Training, Validation and Testing. The training data will be used to train the deep learning CNN model and its parameters will be fine tuned with the validation data. Finally, the performance of the data will be evaluated using the test data(data the model has not previously seen).</p>

In [5]:
# Separate in train and test data
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=1)

# 加载截断图像的容错设置
ImageFile.LOAD_TRUNCATED_IMAGES = True

# 检查并移除损坏的图像文件，同时更新 DataFrame
def check_images(dataframe, filepath_col='Filepath'):
    valid_files = []
    for filepath in dataframe[filepath_col]:
        if not os.path.exists(filepath):
            print(f"File not found, skipping: {filepath}")
            continue
        try:
            # 使用 with 语句确保文件在检查后关闭
            with Image.open(filepath) as img:
                img.verify()  # 检查图像是否可以正常打开
            valid_files.append(filepath)  # 记录有效文件
        except (IOError, SyntaxError, OSError):
            print(f"Corrupted file detected and removed: {filepath}")
            try:
                os.remove(filepath)  # 删除损坏的文件
            except PermissionError:
                print(f"Could not delete {filepath} as it is being used by another process.")
    
    # 更新 DataFrame，仅保留有效文件
    return dataframe[dataframe[filepath_col].isin(valid_files)].reset_index(drop=True)

# 对训练和测试数据进行检查并更新 DataFrame
train_df = check_images(train_df)
test_df = check_images(test_df)
# 对训练和测试数据进行检查
check_images(train_df)
check_images(test_df)


C:\Users\Anderson\.conda\envs\COMP9444proj\lib\site-packages\PIL\TiffImagePlugin.py:864: UserWarning: Truncated File Read
  warnings.warn(str(msg))


,Filepath,Label
0,..\yoga_dataset_links\Gate_Pose_or_Parighasana...,Gate_Pose_or_Parighasana_
1,..\yoga_dataset_links\Split pose\2_0_20.jpg,Split pose
2,..\yoga_dataset_links\Standing_Split_pose_or_U...,Standing_Split_pose_or_Urdhva_Prasarita_Eka_Pa...
3,..\yoga_dataset_links\Reclining_Hand-to-Big-To...,Reclining_Hand-to-Big-Toe_Pose_or_Supta_Padang...
4,..\yoga_dataset_links\Plow_Pose_or_Halasana_\0...,Plow_Pose_or_Halasana_
...,...,...
7435,..\yoga_dataset_links\Bharadvaja's_Twist_pose_...,Bharadvaja's_Twist_pose_or_Bharadvajasana_I_
7436,..\yoga_dataset_links\Bow_Pose_or_Dhanurasana_...,Bow_Pose_or_Dhanurasana_
7437,..\yoga_dataset_links\Supta_Virasana_Vajrasana...,Supta_Virasana_Vajrasana
7438,..\yoga_dataset_links\Firefly_Pose_or_Tittibha...,Firefly_Pose_or_Tittibhasana_


In [6]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v3.preprocess_input
)

In [7]:
# Split the data into three categories.
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=64,
    shuffle=False
)

Found 23808 validated image filenames belonging to 82 classes.
Found 5952 validated image filenames belonging to 82 classes.
Found 7440 validated image filenames belonging to 82 classes.


In [8]:
# Resize Layer
resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224,224),
  layers.experimental.preprocessing.Rescaling(1./255),
])

# Setup data augmentation
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal_and_vertical"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),                       
], name="data_augmentation")

### 1.10 Model Evaluation
<p style="font-size:15px; font-family:verdana; line-height: 1.7em">The test dataset will be used to evaluate the performance of the model.One of the metrics that will be tested would be accuracy which measures the fraction of predictions the model got right. Other metrics are as follows:   </p>

<h3>Precision(P):</h3> 
<p style="font-size:15px; font-family:verdana; line-height: 1.7em">The fraction of true positives (TP, correct predictions) from the total amount of relevant results, i.e., the sum of TP and false positives (FP). For multi-class classification problems, P is averaged among the classes. The following is the formula for precision.</p>

<h4> <center>$P=TP/(TP+FP)$</center></h4> 

<h3>Recall(R): </h3> 
<p style="font-size:15px; font-family:verdana; line-height: 1.7em">The fraction of TP from the total amount of TP and false negatives (FN). For multi-class classification problems, R gets averaged among all the classes. The following is the formula for recall.</p>
<h4><center>$R=TP/(TP+FN)$</center></h4>

<h3>F1 score(F1): </h3>

<p style="font-size:15px; font-family:verdana; line-height: 1.7em">The harmonic mean of precision and recall. For multi-class classification problems, F1 gets averaged among all the classes. The following is the formula for F1 score.</p>
<h4><center>$F1=2 * (TP * FP)/(TP+FP)$</center></h4>


In [9]:
# 加载完整模型
model = tf.keras.models.load_model("yoga_classification_complete_model.h5")
print("load complete")

results = model.evaluate(test_images, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

OSError: No file or directory found at yoga_classification_complete_model.h5

### 1.11 Visualizing loss curves

In [ ]:
# plot_loss_curves(history)

### 1.12 Making predictions on the test data

In [ ]:
# Predict the label of the test_images
pred = model.predict(test_images)
pred = np.argmax(pred,axis=1)

# Map the label
labels = (train_images.class_indices)
labels = dict((v,k) for k,v in labels.items())
pred = [labels[k] for k in pred]

# Display the result
print(f'The first 5 predictions: {pred[:5]}')

In [ ]:
  # Display 15 random pictures from the dataset with their labels
random_index = np.random.randint(0, len(test_df) - 1, 15)
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(test_df.Filepath.iloc[random_index[i]]))
    if test_df.Label.iloc[random_index[i]] == pred[random_index[i]]:
      color = "green"
    else:
      color = "red"
    ax.set_title(f"True: {test_df.Label.iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}", color=color)
plt.show()
plt.tight_layout()

### 1.13 Plotting the classification reports and confusion matrix

In [ ]:
y_test = list(test_df.Label)
print(classification_report(y_test, pred))

report = classification_report(y_test, pred, output_dict=True)
df = pd.DataFrame(report).transpose()
df

### 1.14 Grad-Cam Visualization  - Class Activation Map

In [ ]:
def get_img_array(img_path, size=(224, 224)):
    # 加载图像并调整大小
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
    array = tf.keras.preprocessing.image.img_to_array(img)
    # 添加一个维度，将图像转换为批量大小
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # 创建一个模型，将输入图像映射到最后一个卷积层的激活和输出预测
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # 计算输入图像的梯度
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    
    # 计算相对于最后一个卷积层输出特征图的梯度
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # 获取每个特征通道的梯度平均值
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # 生成热力图
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # 归一化热力图
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
    # 加载原始图像
    img = tf.keras.preprocessing.image.load_img(img_path)
    img = tf.keras.preprocessing.image.img_to_array(img)

    # 将热力图缩放到 0-255
    heatmap = np.uint8(255 * heatmap)

    # 使用 jet 色图来着色热力图
    jet = cm.get_cmap("jet")

    # 获取 RGB 值
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # 将彩色热力图转换为图像
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # 将热力图叠加到原始图像
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    
    # 保存并返回叠加后的图像
    superimposed_img.save(cam_path)
    return cam_path

# 配置模型的预处理
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions

# 设定模型最后一个卷积层名称和图像大小
last_conv_layer_name = "Conv_1"  # 更新为实际的最后卷积层名称
img_size = (224, 224)

# 移除最后一层的 softmax
model = tf.keras.applications.MobileNetV2(weights="imagenet", include_top=True)
model.layers[-1].activation = None

# 随机选择测试集的图像并生成 Grad-CAM
fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(15, 10), subplot_kw={'xticks': [], 'yticks': []})
random_index = random.sample(range(len(test_df)), 15)  # 假设 test_df 是你的测试集 DataFrame

for i, ax in enumerate(axes.flat):
    img_path = test_df.Filepath.iloc[random_index[i]]
    img_array = preprocess_input(get_img_array(img_path, size=img_size))
    heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
    cam_path = save_and_display_gradcam(img_path, heatmap)
    ax.imshow(plt.imread(cam_path))
    ax.set_title(f"True: {test_df.Label.iloc[random_index[i]]}\nPredicted: {pred[random_index[i]]}")

plt.tight_layout()
plt.show()

In [ ]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
import random

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
from PIL import Image, ImageFile

# Tensorflow Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("Using GPU:", tf.config.list_physical_devices('GPU')[0])
else:
    print("No GPU detected, using CPU instead.")

# System libraries
from pathlib import Path
import os.path
import requests
import sys
import os

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

# Data loading and preprocessing
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)

# Changed: Aligned data generator with training settings
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2,
    rescale=1./255  # Changed: using same preprocessing as training
)

test_generator = ImageDataGenerator(
    rescale=1./255  # Changed: using same preprocessing as training
)

# Changed: Using directory-based data loading like in training
dataset_path = "../yoga_dataset_links_skeleton"  # Update with your path

# Changed: Removed DataFrame-based loading and using directory structure directly
train_ds = data_generator.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training',
    shuffle=True,
    seed=42
)

val_ds = data_generator.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

# Load model
model = tf.keras.models.load_model("final/yoga_pose_model_final.h5")  # Update with your model path
print("Model loaded successfully")

# 在加载模型后添加以下代码
def plot_training_curves():
    # Loss曲线
    plt.figure(figsize=(8, 6))
    plt.plot(history['loss'], label='training_loss')
    plt.plot(history['val_loss'], label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Accuracy曲线
    plt.figure(figsize=(8, 6))
    plt.plot(history['accuracy'], label='training_accuracy')
    plt.plot(history['val_accuracy'], label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    plt.show()

# 加载训练历史
try:
    history = np.load('training_history.npy', allow_pickle=True).item()
    plot_training_curves()
except:
    print("Training history file not found")


# Evaluate model
results = model.evaluate(val_ds, verbose=0)
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

# Predict
predictions = model.predict(val_ds)
pred_indices = np.argmax(predictions, axis=1)

# Get class names directly from directory structure to maintain same order
class_names = sorted(os.listdir(dataset_path))  # Changed: Getting classes in same order as training
pred = [class_names[idx] for idx in pred_indices]

print(f'The first 5 predictions: {pred[:5]}')

# Get true labels
true_labels = []
filenames = val_ds.filenames
for filename in filenames:
    class_name = os.path.split(os.path.dirname(filename))[1]
    true_labels.append(class_name)

# Classification Report
print("\nClassification Report:")
print(classification_report(true_labels, pred))

# Display Results
# Changed: Using directory structure to get images
image_files = [os.path.join(dataset_path, true_labels[i], os.path.basename(filenames[i])) 
               for i in range(len(filenames))]
random_indices = np.random.randint(0, len(image_files), 15)

fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    img_path = image_files[random_indices[i]]
    ax.imshow(plt.imread(img_path))
    color = "green" if true_labels[random_indices[i]] == pred[random_indices[i]] else "red"
    ax.set_title(f"True: {true_labels[random_indices[i]]}\nPredicted: {pred[random_indices[i]]}", 
                color=color)

plt.tight_layout()
plt.show()

# Save important metrics
results_df = pd.DataFrame({
    'Metric': ['Test Loss', 'Test Accuracy'],
    'Value': [results[0], results[1]]
})
results_df.to_csv('evaluation_results.csv', index=False)

In [ ]:
# Import Data Science Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import itertools
import random

# Import visualization libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cv2
from PIL import Image, ImageFile

# Tensorflow Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import Model
from tensorflow.keras.layers.experimental import preprocessing

# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
if tf.config.list_physical_devices('GPU'):
    print("Using GPU:", tf.config.list_physical_devices('GPU')[0])
else:
    print("No GPU detected, using CPU instead.")

# System libraries
from pathlib import Path
import os.path
import requests
import sys
import os

# Metrics
from sklearn.metrics import classification_report, confusion_matrix

# Data loading and preprocessing
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)

# Changed: Aligned data generator with training settings
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2,
    rescale=1./255  # Changed: using same preprocessing as training
)

test_generator = ImageDataGenerator(
    rescale=1./255  # Changed: using same preprocessing as training
)

# Changed: Using directory-based data loading like in training
dataset_path = "../yoga_dataset_links_skeleton"  # Update with your path

# Changed: Removed DataFrame-based loading and using directory structure directly
train_ds = data_generator.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='training',
    shuffle=True,
    seed=42
)

val_ds = data_generator.flow_from_directory(
    dataset_path,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

def print_classification_report(report):
    # 打印头部
    print("\nClassification Report:")
    print(" " * 75 + "precision    recall  f1-score   support")
    print()
    
    # 按行处理报告
    for line in report.split('\n'):
        if not line.strip():  # 跳过空行
            continue
            
        if line.lstrip().startswith('accuracy'):
            name = 'accuracy'
            line = line[8:]  # 移除 "accuracy"
            print(" " * 75 + name + line)
        elif line.lstrip().startswith('macro'):
            name = 'macro avg'
            line = line[21:]  # 移除 "macro avg"
            print(" " * 75 + name + line)
        elif line.lstrip().startswith('weighted'):
            name = 'weighted avg'
            line = line[21:]  # 移除 "weighted avg"
            print(" " * 75 + name + line)
        elif not line.startswith(' ' * 10):  # 跳过原始表头行
            try:
                # 处理常规类别行
                items = line.split()
                if len(items) >= 5:  # 确保行有足够的元素
                    metrics = items[-4:]  # 最后4个元素是指标
                    name = " ".join(items[:-4])  # 其余的是类名
                    metrics_str = "{:>9} {:>9} {:>9} {:>9}".format(*metrics)
                    print(f"{name:>75} {metrics_str}")
            except Exception as e:
                continue  # 如果有任何处理错误，跳过该行

# Load model
model = tf.keras.models.load_model("final/yoga_pose_model_final.h5")  # Update with your model path
print("Model loaded successfully")

# Evaluate model
results = model.evaluate(val_ds, verbose=0)
print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

# Predict
predictions = model.predict(val_ds)
pred_indices = np.argmax(predictions, axis=1)

# Get class names directly from directory structure to maintain same order
class_names = sorted(os.listdir(dataset_path))  # Changed: Getting classes in same order as training
pred = [class_names[idx] for idx in pred_indices]

print(f'The first 5 predictions: {pred[:5]}')

# Get true labels
true_labels = []
filenames = val_ds.filenames
for filename in filenames:
    class_name = os.path.split(os.path.dirname(filename))[1]
    true_labels.append(class_name)

# Generate classification report with proper formatting
report = classification_report(true_labels, pred)
print_classification_report(report)

# Display Results
# Changed: Using directory structure to get images
image_files = [os.path.join(dataset_path, true_labels[i], os.path.basename(filenames[i])) 
               for i in range(len(filenames))]
random_indices = np.random.randint(0, len(image_files), 15)

fig, axes = plt.subplots(nrows=3, ncols=5, figsize=(25, 15),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    img_path = image_files[random_indices[i]]
    ax.imshow(plt.imread(img_path))
    color = "green" if true_labels[random_indices[i]] == pred[random_indices[i]] else "red"
    ax.set_title(f"True: {true_labels[random_indices[i]]}\nPredicted: {pred[random_indices[i]]}", 
                color=color)

plt.tight_layout()
plt.show()

# Save important metrics
results_df = pd.DataFrame({
    'Metric': ['Test Loss', 'Test Accuracy'],
    'Value': [results[0], results[1]]
})
results_df.to_csv('evaluation_results.csv', index=False)